# code-book rules for the "default ruleset for Java, by r2c" 

There are 28 rules in this ruleset. In the notebook, you'll find 
parallel implementations of each of the rules for `code-book`. This is
to help me get an idea of what our high level API should look like, what
kinds of operations we need to support, and where we might be able to
"do better" than SemGrep. This also serves as a nice comparison for
performance (eventually, will need to set SemGrep up on the same data).

In [8]:
!pip3 install regex

distutils: /usr/local/include/python3.9/UNKNOWN
sysconfig: /usr/local/include/python3.9
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 730 kB 4.0 MB/s eta 0:00:01
distutils: /usr/local/include/python3.9/UNKNOWN
sysconfig: /usr/local/include/python3.9
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:

temp1 = utils.query_java(
    'type_identifier',
    '"HttpServletRequest"=type_identifier{=1}formal_parameter'
)

temp2 = utils.query_java(
    'formal_parameter_ref',
    'formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_child{=2}object_creation_expression'
)

m1 = utils.merge_paths(temp1, temp2, on=('gids.2', 'defs.1'))

for res in m1.values_r:
    print(utils.get_text_from_capture(res, 3))


temp3 = utils.query_java(
    'formal_parameter_ref',
    'formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_value{=1}variable_declarator'
)

temp4 = utils.query_java(
    'variable_declarator_ref',
    'variable_declarator_ref.f_child{=2}object_creation_expression{}method_declaration'
)

m2_1 = utils.merge_paths(temp1, temp3, on=('gids.2', 'defs.1'))
m2_2 = utils.merge_paths(m2_1.values_r, temp4, on=('gids.3', 'defs.1'))

for res in m2_2.values_r:
    print('// The match ----------------------------------------- ')
    print(utils.get_text_from_capture(res, 2))
    print('// Context ------------------------------------------- ')
    try:
        print(utils.get_text_from_capture(res, 3))
    except:
        print('// Error encoding file (ascii codec can\'t encode...)')


Preapring query...
  + Prepared: 0.0018s
Running query...
  + Ran `"HttpServletRequest"=type_identifier{=1}formal_parameter`: 5.4243s
Preapring query...
  + Prepared: 0.0013s
Running query...
  + Ran `formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_child{=2}object_creation_expression`: 2.1497s
Preapring query...
  + Prepared: 0.0015s
Running query...
  + Ran `formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_value{=1}variable_declarator`: 1.9186s
Preapring query...
  + Prepared: 0.0013s
Running query...
  + Ran `variable_declarator_ref.f_child{=2}object_creation_expression{}method_declaration`: 6.1524s
// The match ----------------------------------------- 
new URI(imageURIString)
// Context ------------------------------------------- 
@Override
  protected void doGet(HttpServletRequest request,
                       HttpServletResponse response) throws ServletException, IOException {

    String imageURIString = request.getParameter("u");
    if 

In [72]:
class CodeBookJava:
  def __init__(self):
    self.paths = {}
    self.merges = {}
    self.merge_idx = 0
    self.type_idx = 0
    self.primary_type = None

  def _set_primary(self, the_type):
    self.type_idx += 1
    self.primary_type = the_type + '.{}'.format(self.type_idx)
    self.paths[self.primary_type] = the_type
   
  def _primary_type_is(self, the_type):
    return self.primary_type.split('.')[0] == the_type
  
  def _add_prefix(self, prefix):
    self.paths[self.primary_type] = prefix + self.paths[self.primary_type]
   
  def _add_suffix(self, suffix):
    self.paths[self.primary_type] = self.paths[self.primary_type] + suffix
    
  def _clone(self):
    from copy import deepcopy
    return deepcopy(self)
   
  def _add_merge(self, left, right, lkey, rkey):
    self.merge_idx += 1
    self.merges['m{}'.format(self.merge_idx)] = (
      left, right, lkey, rkey
    )
   
  def _combine_paths(self, other):
    self.paths.update(other.paths)
  
  def new(self, name=None):
    self._set_primary('object_creation_expression')
    if name is not None:
      self._add_prefix('"{}"='.format(name))
    return self
 
  def any_arg_is(self, other):
    if self._primary_type_is('object_creation_expression') and type(other) == list:
      for item in other:
        item._add_suffix('.f_child{=2}')
        item._add_suffix(self.paths[self.primary_type])
      return other
    return self
  
  def calls(self, name=None):
    self._set_primary('method_invocation')
    if name is not None:
      self._add_prefix('"{}"='.format(name))
    return self
 
  def receiver_is(self, other):
    self._combine_paths(other)
    if other._primary_type_is('formal_parameter') and self._primary_type_is('method_invocation'):
      self._add_prefix('formal_parameter_ref.f_object{=1}')
      self._add_merge(other.primary_type, self.primary_type, 'gids.2', 'defs.1')
    return self

  def ref(self):
    if self._primary_type_is('method_invocation'):
      # We can ref direct, or via variable
      copy = self._clone()
      copy._add_suffix('.f_value{=1}variable_declarator')
      copy._set_primary('variable_declarator_ref')
      copy._add_merge(self.primary_type, copy.primary_type, 'gids.3', 'defs.1')
      return [ self, copy ]
      
    return self
  
  def method_params(self, name=None, type=None):
    self._set_primary('formal_parameter')
    if name is not None:
      self._add_prefix('"{}"='.format(name))
    if type is not None:
      self._add_prefix('"{}"=type_identifier{{=1}}'.format(type))
    return self
  
  def _execute(self):
    for key, (lref, rref, lkey, rkey) in self.merges.items():
      print('t1 = query_java("{}", \'{}\')'.format(lref.split('.')[0], self.paths[lref]))
      print('t2 = query_java("{}", \'{}\')'.format(rref.split('.')[0], self.paths[rref]))
      print('merge_paths(t1, t2, on=("{}", "{}")'.format(lkey, rkey))
    return []
 

def execute_all(queries):
  for query in queries:
    yield from query._execute()

cb = lambda: CodeBookJava()

In [73]:
# SemGrep: httpservlet-path-traversal

call_to_get_param = cb().calls('getParameter').receiver_is(
    cb().method_params(type='HttpServletRequest')
)

# matches = execute_all(cb().new('File').any_arg_is(call_to_get_param.ref()))
matches = list(execute_all(
  cb().new().any_arg_is(call_to_get_param.ref())
))

t1 = query_java("formal_parameter", '"HttpServletRequest"=type_identifier{=1}formal_parameter')
t2 = query_java("method_invocation", 'formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_child{=2}object_creation_expression')
merge_paths(t1, t2, on=("gids.2", "defs.1")
t1 = query_java("formal_parameter", '"HttpServletRequest"=type_identifier{=1}formal_parameter')
t2 = query_java("method_invocation", 'formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_value{=1}variable_declarator')
merge_paths(t1, t2, on=("gids.2", "defs.1")
t1 = query_java("method_invocation", 'formal_parameter_ref.f_object{=1}"getParameter"=method_invocation.f_value{=1}variable_declarator')
t2 = query_java("variable_declarator_ref", 'variable_declarator_ref.f_child{=2}object_creation_expression')
merge_paths(t1, t2, on=("gids.3", "defs.1")


In [ ]:
# SemGrep: servletresponse-writer-xss

call_to_get_param = cb.calls('getParameter').receiver_is(
    cb.method_params(type='HttpServletRequest')
).bind()

call_to_get_writer = cb.calls('getWriter').receiver_is(
    cb.method_params(type='HttpServletResponse')
).bind()

matches = (
    cb.calls('write')
      .any_arg_is(call_to_get_param)
      .receiver_is(call_to_get_writer)
)


In [ ]:
# SemGrep: anonymous-ldap-bind

bad_env = cb.calls('put').first_arg_is(
    cb.field_ref(name='SECURITY_AUTHENTICATION')
).second_arg_is(cb.str('none')).bind_receiver()

matches = cb.new('InitialDirContext').first_arg_is(bad_env)


In [ ]:
# SemGrep: bad-hexa-conversion

digest_results = cb.calls('digest').receiver_is(
    cb.vars(type='MessageDigest').bind()
).bind()

for_over_results = cb.fors().target_container(digest_results).bind()

matches = cb.calls('Integer.toHexString').any_arg_is(
    cb.deep_ref(for_over_results)
)

In [ ]:
# SemGrep: cbc-padding-oracle

matches = cb.calls('getInstance').first_arg_is(
    cb.str(regex=r".*/CBC/PKCS5Padding/")
)


In [ ]:
# SemGrep: command-injection-formatted-runtime-call

# This one is tricky! Trying to say no exec( ... "sh", "-c", user_supplied, ...)

matches1 = cb.calls(['exec', 'loadLibrary']).first_arg_is(
    cb.str_concat_or_format()
).receiver_is(cb.calls('getRuntime').bind())

matches2 = cb.calls('exec').any_arg_is(
    cb.deep_ref(cb.siblings(
        cb.str(regex=r"(sh|bash|ksh|csh|tcsh|zsh)"),
        cb.str('-c'),
        cb.var().has_no_init()
    ))
)


In [ ]:
# SemGrep: formatted-sql-string

# TODO: this one is also quite complex (just long...)
# we can probably make it a lot shorter!

# https://semgrep.dev/editor?registry=java.lang.security.audit.formatted-sql-string.formatted-sql-string

In [ ]:
# SemGrep: http-response-splitting

bad_cookie1 = cb.new('Cookie').any_arg_is(
    cb.calls('getParameter').bind()
)

bad_cookie2 = cb.new('Cookie').any_arg_is(
    cb.method_params().annotated_with('@PathVariable').bind()
)

matches = cb.calls('addCookie').first_arg_is(
    cb.either(bad_cookie1, bad_cookie2)
)



In [ ]:
# SemGrep: ldap-injection

context_var = cb.var([
    'InitialDirContext',
    'DirContext',
    'InitialLdapContext',
    'LdapContext',
    'LdapCtx',
    'EventDirContext'
]).bind()

matches = cb.calls('search').receiver_is(
    context_var
).second_arg_is(
    cb.anything_but(cb.str())
)


In [ ]:
# SemGrep: object-deserialization

matches = cb.new('ObjectInputStream')


In [ ]:
# SemGrep: script-engine-injection

matches = cb.calls('eval').receiver_is(cb.either(
    cb.field(type='ScriptEngine').bind(),
    cb.var(type='ScriptEngine').bind()
)).first_arg_is(
    cb.anything_but(cb.str())
)
